
This script processes product data from an online store using the following libraries:

1. `pandas` is used for handling the data in tabular form.
2. `requests` is used to retrieve web pages.
3. `BeautifulSoup` is used to parse HTML and extract specific elements.

The main function, `get_price_by_url_ml`, retrieves the product price and name from the given URL and returns it in a structured format.


In [2]:
from bs4 import BeautifulSoup # To parse HTML and XML documents
import pandas as pd # For data manipulation and analysis
import requests # To send HTTP requests

In [16]:
def get_price_by_url_ml(url : str):  
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser" )
    try:
        title = soup.find("h1", class_="ui-pdp-title").text.strip() # type: ignore
    except AttributeError as e:
        title = ""
    try:
        price_product_integer = soup.find("div", class_="ui-pdp-price__second-line").find("span", class_="andes-money-amount__fraction").text # type: ignore
    except AttributeError as e:
        price_product_integer = "0"
    try:
        price_product_decimal = soup.find("div", class_="ui-pdp-price__second-line").find("span", class_="andes-money-amount__cents andes-money-amount__cents--superscript-36").text # type: ignore
    except AttributeError as e:
        price_product_decimal = "0"
    price_product_final = int(price_product_integer.replace(".","")) + float(price_product_decimal)/100
    #print(f"producto: {title} \nprecio: {price_product_final}") 
    return {"product": title.replace("\xa0","") , "price" : price_product_final }

In [22]:
url = "https://articulo.mercadolibre.com.ar/MLA-872773414-natufarma-ginkgo-con-vitamina-b1-x-60-comprimidos-_JM#polycard_client=search-nordic&position=26&search_layout=stack&type=item&tracking_id=7d83fa3c-d0b7-4cbb-8332-a9a5ec54b964"
get_price_by_url_ml(url=url)

{'product': 'Natufarma Ginkgo Con Vitamina B1 X 60 Comprimidos',
 'price': 8415.58}

In [18]:
def read_file_excel(path: str, sheet_name: str):
    
    """
    Reads an Excel file and returns its contents as a DataFrame.

    Parameters:
    path (str): The file path to the Excel file.
    sheet_name (str): The name of the sheet to read.

    Returns:
    DataFrame: A pandas DataFrame containing the data from the specified sheet, 
               or None if an error occurs.

    Exceptions:
    - FileNotFoundError: Raised if the file at the specified path is not found.
    - ValueError: Raised if the specified sheet does not exist in the file.
    - Exception: Catches any other exceptions and prints the error message.
    """
    try:
        df = pd.read_excel(path, sheet_name=sheet_name,header=None)
        return df
    except FileNotFoundError:
        print(f"Error: The file at {path} was not found.")
    except ValueError:
        print(f"Error: The sheet {sheet_name} does not exist in the file.")
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [23]:
def read_file_excel_get_prices(path: str, sheet_name: str):

    """
    Reads an Excel file, retrieves prices from URLs in the specified sheet, 
    and returns a DataFrame with the results.

    Parameters:
    path (str): The path to the Excel file.
    sheet_name (str): The name of the sheet to read.

    Returns:
    DataFrame: A pandas DataFrame containing the prices, title and associated URLs.

    Exceptions:
    - Prints errors if the file, sheet, or URL processing encounters issues.
    """

    df = None
    list_price_name = []
    try:
        df = read_file_excel(path=path, sheet_name=sheet_name)
    except Exception as e:
        print(e)
        return
    try:
        if df is not None and not df.empty: 
            for index, row in df.iterrows():
                url = row[0]
                data = get_price_by_url_ml(url=url)
                data["url"] = url
                list_price_name.append(data)
    except Exception as e:
        print(e)
        return 
    df_price_name = pd.DataFrame(list_price_name)
    return df_price_name
        
df = read_file_excel_get_prices("products.xlsx", sheet_name="products")


In [24]:
if df is not None:
    df.to_excel('price_url_title.xlsx', index=False)

In [25]:
df

,product,price,url
0,Honda Cb 300 Twister Entrega Ya Fcia Tornado W...,6989330.00,https://moto.mercadolibre.com.ar/MLA-138565162...
1,Honda Xr 250 Tornado Financia Xr Wave Glh Twis...,7010700.00,https://moto.mercadolibre.com.ar/MLA-141166341...
2,Honda Cb 300 Twister Entrega Ya Fcia Tornado W...,6989330.00,https://moto.mercadolibre.com.ar/MLA-138565162...
3,¡suzuki Gixxer 150 Inyeccion Suzuki Agronomia Fi,4945000.00,https://moto.mercadolibre.com.ar/MLA-140938223...
4,Suzuki Agronomia- Suzuki Gsx 125-r Mejor Preci...,3065000.00,https://moto.mercadolibre.com.ar/MLA-160524068...
5,Moto Tvs Rtr 200 Fi Inyeccion No Rouser Agro...,5413755.00,https://moto.mercadolibre.com.ar/MLA-175017899...
6,Tvs Apache Rr310 Abs Agronomia,11788200.00,https://moto.mercadolibre.com.ar/MLA-144193244...
7,Motomel Cg 150 S2 Full Linea 2024 Usb-disco-ta...,2249000.00,https://moto.mercadolibre.com.ar/MLA-167686387...
8,Moto Beta Zontes G1 155 Scrambler Fi Abs,4207000.00,https://moto.mercadolibre.com.ar/MLA-153258259...
9,Zontes R2 350 0km Monobrazo Abs Usb Pantalla T...,6915000.00,https://moto.mercadolibre.com.ar/MLA-179913822...
